# Worksheet 05

Name:  Yu Han <br>
UID: U74808634

### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

1. For each point in the dataset, if the distance to 0 is smaller than to 2, then set the point to cluster A, otherwise, set it to cluster B.
2. Find the mean of each cluster, and set the two mean values as the new centroids.
3. Repeat what we did in step 1 and step 2, untill the centroids stay the same.

b) Describe in plain english what the cost function for k means is.

It is the sum of squared distances from clusters to the centroid. The mean is the centroid where the distance between the centroid and each point from the cluster is the same.

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

K-means seeks to minimize the sum of squared distances between data points and their respective cluster centroids. Since it relies on iterative optimization, it can sometimes converge to a local minimum instead of the global minimum.

d) Does Lloyd's Algorithm always converge? Why / why not?

Yes. The algorithm iteratively assigns data points to the nearest cluster centroid and updates the centroids. The sum of squared distance decreases in each iteration. Since the sum of squared distances is a non-increasing function during each iteration, the graph concaves up which means it always has a minimum.

e) Follow along in class the implementation of Kmeans

In [1]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))

    def initialize(self):

        return np.array([list(self.data[np.random.choice(len(self.data), replace = False)]) for _ in range(self.k)])

    def assign(self, centers):

        self.assignment = np.argmin([np.linalg.norm(centers - x, axis = 1) for  x in self.data], axis = 1)
        
    def get_centers(self):
        
        return np.array([np.mean(self.data[self.assignment == i], axis = 0) for i in range(self.k)])

    def is_diff_clusters(self, centers, new_centers):

        return ((np.linalg.norm(centers - new_centers) > 0).sum() > 0)


    def lloyds(self):

        centers = self.initialize()
        self.assign(centers)
        self.snap(centers)
        new_centers = self.get_centers()

        while self.is_diff_clusters(centers, new_centers):    

            self.assign(new_centers)      
            centers = new_centers    
            self.snap(new_centers)
            new_centers = self.get_centers()
        
        return
            

kmeans = KMeans(X, 6)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)